In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
sns.set()
%matplotlib inline
#sns.set_palette("Set1")
#sns.set_style("whitegrid")
import warnings
warnings.filterwarnings("ignore")

# (Section 1) Before neural networks: Logistic regression

## 1.1 The iris dataset

In [ ]:
iris = sns.load_dataset("iris")

In [ ]:
iris.head()

In [ ]:
iris.shape

In [ ]:
iris.species.value_counts()

In [ ]:
sns.pairplot(iris, hue="species")

In [ ]:
def plot_histogram(my_species,feature,n_bins=20,normed=None,data=iris):
    """Plot a histogram of the given feature according to a given species."""
    mask = data['species']==my_species
    hist0 = data[feature][~mask]
    hist1 = data[feature][mask]
    histograms = [hist0,hist1]
    labels = ['not '+my_species, my_species]
    fig,ax = plt.subplots(figsize=(12,5))
    plt.hist(histograms, histtype='bar',bins=n_bins,normed=normed,label=labels);
    plt.title(my_species,fontsize='xx-large')
    plt.xlabel(feature,fontsize='x-large')
    plt.legend(fontsize='x-large')
    ax.tick_params(labelsize='large')

In [ ]:
plot_histogram('virginica','petal_width')

# Problem we want to address: knowing the petal width, predict if the iris is a virginica

## 1.2 Logistic regression with sklearn: guessing virginica knowing the petal width

In [ ]:
import sklearn.linear_model

In [ ]:
x = iris['petal_width'].reshape(-1,1)

In [ ]:
y = iris.species.apply(lambda x: 1 if x=='virginica' else 0)

In [ ]:
model_sklearn = sklearn.linear_model.LogisticRegression()

In [ ]:
model_sklearn.fit(x,y)

In [ ]:
plot_histogram('virginica','petal_width',normed=True)
petal_widths = np.arange(0,3,0.01)
predicted_proba = model_sklearn.predict_proba(petal_widths.reshape(-1,1))[:,1]
plt.plot(petal_widths,predicted_proba,'r--',label='predicted proba')
plt.plot([0,3],[.5,.5],'k--',linewidth=1)
plt.legend(fontsize='x-large')
plt.yticks([0,0.5,1]);

The accuracy of a model evaluated on the sample x,y is defined as 

$$ \text{Accuracy} = \frac{\text{number of samples correctly classified}}{\text{number of samples}}$$

With keras, we can obtain the accuracy using the score method.

In [ ]:
model_sklearn.score(x,y)

# Keras

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import regularizers
from keras import optimizers

In [ ]:
model_keras = Sequential()

In [ ]:
layer0 = Dense(1,input_dim=1,kernel_regularizer=regularizers.l2(.1))
model_keras.add(layer0)

In [ ]:
layer1 = Activation('sigmoid')
model_keras.add(layer1)

In [ ]:
model_keras.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
def initialize_keras(reg = .1):
    model_keras = Sequential()
    layer0 = Dense(1,input_dim=1,kernel_regularizer=regularizers.l2(reg))
    model_keras.add(layer0)
    layer1 = Activation('sigmoid')
    model_keras.add(layer1)
    sgd = optimizers.SGD(lr=0.05)
    model_keras.compile(optimizer=sgd, loss='binary_crossentropy',metrics=['accuracy'])
    return model_keras

In [ ]:
model_keras.fit(x,y,epochs=100)

In [ ]:
model_keras.metrics_names

In [ ]:
model_keras.evaluate(x,y)

In [ ]:
model_keras.get_weights()

In [ ]:
model_keras = initialize_keras(reg=.01)
plot_histogram('virginica','petal_width',normed=True)
petal_widths = np.arange(0,3,0.01)
model_keras.fit(x,y,epochs=500,verbose=0)
predicted_proba = model_keras.predict(petal_widths.reshape(-1,1))[:,0]
plt.plot(petal_widths,predicted_proba,'r--',label='proba')
plt.plot([0,3],[.5,.5],'k--',linewidth=1)
plt.legend()
plt.yticks([0,0.5,1]);

## The sigmoid function

Here the logistic regression algorithm does the following: try to find a function 
$$ P: [0,3] \to [0,1]$$
such that for a random $x \in [0,3]$ corresponding to a petal width, $P(x)$ is an estimate (= a guess) of the probability that the plant is a virginica.

The idea of logistic regression is to look for a function of the form:
$$P_{w,b}(x) = \sigma(wx+b)$$
where 
$$\sigma : \mathbb{R} \to [0,1]$$ is the **sigmoid function ** defined by 
$$ \sigma(x) = \frac{1}{1+e^{-x}}$$
We call $w$ the **weight** and $b$  the **bias**.

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [ ]:
x_sigmoid = np.arange(-10,10,0.01)
y_sigmoid = sigmoid(x_sigmoid)
plt.plot(x_sigmoid,y_sigmoid)

In [ ]:
plt.figure(figsize=(15,10))

plt.subplot(221)
w=1
b=0
y_shifted = sigmoid(w*x_sigmoid + b)
plt.plot(x_sigmoid,y_shifted,label='w={}\nb={}'.format(w,b))
plt.legend(fontsize = 'xx-large')
#plt.title('Shifted sigmoid')


plt.subplot(222)
w = -1
b = 0
y_shifted = sigmoid(w*x_sigmoid + b)
plt.plot(x_sigmoid,y_shifted,label='w={}\nb={}'.format(w,b))
plt.legend(fontsize = 'xx-large')

plt.subplot(223)
w = 4
b = 0
y_shifted = sigmoid(w*x_sigmoid + b)
plt.plot(x_sigmoid,y_shifted,label='w={}\nb={}'.format(w,b))
plt.legend(fontsize = 'xx-large')

plt.subplot(224)
w = -1
b = 5
y_shifted = sigmoid(w*x_sigmoid + b)
plt.plot(x_sigmoid,y_shifted,label='w={}\nb={}'.format(w,b))
plt.legend(fontsize = 'xx-large')

plt.tight_layout()

Ajouter graphe du reseau neronal

Definir accuracy

# (Section 2) Gradient descent

# Optimization, gradient descent with Keras

In [ ]:
model = Sequential()
model.add(Dense(1,input_dim=1,kernel_regularizer=regularizers.l2(.1)))
model.add(Activation('sigmoid'))
sgd = optimizers.SGD(lr=0.05)
model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
def get_meshgrid(xstart,xend,xstep,ystart,yend,ystep):
    xx,yy = np.mgrid[xstart:xend:xstep,ystart:yend:ystep]
    return xx,yy
x_grid, y_grid = get_meshgrid(-10,10,.1,-10,10,.1)

In [ ]:
model.metrics_names

In [ ]:
model.evaluate(x,y)

## Plot accuracy

In [ ]:
def get_accuracy(model,weight,bias,x,y,C=1.):
    """Compute the accuracy function."""
    layer =  model.layers[0]
    layer.set_weights( [ np.array([[weight]]) , np.array([bias]) ] ) 
    accuracy = model.evaluate(x,y,verbose=0)[1]
    return accuracy

def get_accuracy1(weigth,bias):
    return get_accuracy(model,weigth,bias,x,y,1.)

vaccuracy = np.vectorize(get_accuracy1)

In [ ]:
#takes two minutes to run
acc_grid = vaccuracy(x_grid,y_grid)

In [ ]:
# import pickle
# with open('../data/acc_grid', 'wb') as f:
#     # Pickle the 'data' dictionary using the highest protocol available.
#     pickle.dump(acc_grid, f)

In [ ]:
# import pickle
# with open('../data/acc_grid', 'rb') as f:
#     # Pickle the 'data' dictionary using the highest protocol available.
#     acc_grid =pickle.load(f)

In [ ]:
fig,ax_acc = plt.subplots(figsize=(15,7))
plt.pcolor(x_grid,y_grid,acc_grid,cmap='RdBu_r')
plt.colorbar()
plt.title('Accuracy',fontsize='xx-large')
plt.xlabel('Weight (w)',fontsize='x-large')
plt.ylabel('Bias (b)',fontsize='x-large')
ax_acc.tick_params(labelsize='large')

There are so many zones where the accuracy are the same that it is impossible to find a method that would improve the weights step by step.

To overcome this problem, we introduce a new metric: the cross entropy which we call our loss function.

## The cross entropy  loss function

Let us fix $w$ and $b$. We define 
$$\mathcal{L}(p,y) = y \log(p) + (1-y)\log(1-p)$$
$$\mathcal{L_{w,b}} = \sum_{i=1}^n y_i \log(p_i) + (1-y_i)\log(1-p_i)$$
where $y_i\in \{0,1\}$ is the actual classe of the i-th sample and $p_i \in [0,1]$ is the probability $P_{w,b}(x_i)$ calculated by the logistic regression model for parameter values $w$ and $b$.

In [ ]:
pp = np.arange(0,1,.01)
loss1 = - np.log(pp)
loss0 = -np.log(1-pp)
plt.plot(pp,loss1,label='y = 1')
plt.plot(pp,loss0,label='y = 0')
plt.xlabel('p',fontsize='xx-large')
plt.ylabel('Loss',fontsize='xx-large')
plt.title('Loss function:  $\mathcal{L}(p,y)$',fontsize='xx-large')
_=plt.legend()

In [ ]:
def initialise_keras_model(initial_weight = 9,initial_bias = 9,lr=0.05,reg=.1):
    model = Sequential()
    model.add(Dense(1,input_dim=1,kernel_regularizer=regularizers.l2(.1)))
    model.add(Activation('sigmoid'))
    sgd = optimizers.SGD(lr=lr)
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
    layer =  model.layers[0]
    layer.set_weights( [ np.array([[initial_weight]]) , np.array([initial_bias]) ] ) 
    return model

In [ ]:
def get_loss(model,weight,bias,x,y,C=1.):
    """Compute the loss function."""
    layer =  model.layers[0]
    layer.set_weights( [ np.array([[weight]]) , np.array([bias]) ] ) 
    loss = model.evaluate(x,y,verbose=0)[0]
    return loss

def get_loss1(weigth,bias):
    return get_loss(model,weigth,bias,x,y,1.)

vloss = np.vectorize(get_loss1)

In [ ]:
loss_grid = vloss(x_grid,y_grid)

In [ ]:
# import pickle
# with open('../data/loss_grid', 'wb') as f:
#     # Pickle the 'data' dictionary using the highest protocol available.
#     pickle.dump(loss_grid, f)

In [ ]:
# import pickle
# with open('../data/loss_grid', 'rb') as f:
#     # Pickle the 'data' dictionary using the highest protocol available.
#     loss_grid =pickle.load(f)

In [ ]:
fig , ax_loss = plt.subplots(figsize=(20,10))
plt.pcolor(x_grid , y_grid , loss_grid , norm=colors.LogNorm() , cmap='RdBu_r')
plt.colorbar()
contour = plt.contour(x_grid, y_grid, loss_grid,20)
plt.title('Loss function',fontsize='xx-large')
plt.xlabel('weight',fontsize='x-large')
plt.ylabel('bias',fontsize='x-large')

model = initialise_keras_model(reg=.05)
steps = 15
for i in range(steps):
    old_weight, old_bias = model.get_weights()[0][0][0], model.get_weights()[1][0]
    model.fit(x,y,verbose=0,epochs = 8)
    weight, bias = model.get_weights()[0][0][0], model.get_weights()[1][0]
    plt.plot([old_weight,weight],[old_bias,bias],'kX--',markersize=8)

# (Section 3) Neural Networks: when linear methods are not sufficient

In [ ]:
plot_histogram('versicolor','petal_width')

## Goal: determine if an iris is a versicolor knowing its petal width

## First let's try using logistic regression

In [ ]:
x = iris.petal_width
y = iris.species.apply(lambda x: 1 if x=='versicolor' else 0)
x_train, x_test, y_train, y_test = \
sklearn.model_selection.train_test_split(x,y,test_size = 0.2,random_state=0)

In [ ]:
model_versicolor = Sequential()
model_versicolor.add(Dense(1, input_dim=1 , kernel_regularizer=regularizers.l2(.1)))
model_versicolor.add(Activation('sigmoid'))
model_versicolor.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_versicolor.fit(x_train,y_train,epochs=1000)

In [ ]:
plot_histogram('versicolor','petal_width',normed=True)
petal_widths = np.arange(0,3,0.01)
probas = model_versicolor.predict(petal_widths.reshape(-1,1))[:,0]
plt.plot(petal_widths,probas,'r--',label='proba')
plt.plot([0,3],[.5,.5],'k--',linewidth=1,label = 'proba = 0.5')
plt.legend()
plt.yticks([0,0.5,1]);

## Now let's add a new layer to the neural network

In [ ]:
def initialize_versicolor_model(lr = .1,h=5):
    model = Sequential()
    model.add(Dense(h, input_dim=1 ) )
    model.add(Activation('sigmoid'))
    model.add(Dense(1, input_dim=1 ) )
    model.add(Activation('sigmoid'))
    sgd = optimizers.SGD(lr=lr)
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model_versicolor = initialize_versicolor_model()

In [ ]:
n_rows = n_cols = 3
plt.subplots(n_rows,n_cols,figsize=(18,15),sharex=True,sharey=True)
epochs=200
petal_widths = np.arange(0,3,0.01)
for i in range(1,n_rows**2+1):
    plt.subplot(n_rows,n_cols, i)
    model_versicolor.fit(x_train,y_train,epochs=epochs,verbose=0)
    probas = model_versicolor.predict(petal_widths.reshape(-1,1))[:,0]
    plt.plot(petal_widths,probas,label='after  {} epochs'.format(i*epochs))
    plt.ylim((0,1))
    plt.legend(fontsize='large')

In [ ]:
plot_histogram('versicolor','petal_width',normed=True)
probas = model_versicolor.predict(petal_widths.reshape(-1,1))[:,0]
plt.plot(petal_widths,probas,'b',label='after  {} epochs'.format(i));